Alunos:

*   Andréa Fonseca
*   Fábio Cardoso
*   Eduardo Leite



In [1]:
!pip install langchain_openai langchain_core langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 796.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found ex

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
llm.invoke("Qual é a capital do Brasil?").content

'A capital do Brasil é Brasília.'

In [5]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve um arquivo de texto"""
    with open(file_name, "w") as f:
        f.write(content)

@tool
def count_letters(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra em uma palavra"""
    count = 0
    for char in word:
        if char == letter:
            count += 1
    return count

tools = [weather, multiply, write_file, count_letters]
llm_with_tools = llm.bind_tools(tools)

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima, nem sabe fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [8]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages( # scratchpad para guardar internamente as respostas
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input": "Qual o clima no Rio de Janeiro?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Rio de Janeiro'}`


nubladoO clima no Rio de Janeiro está nublado.

> Finished chain.


{'input': 'Qual o clima no Rio de Janeiro?',
 'output': 'O clima no Rio de Janeiro está nublado.'}

## Exercícios
Um agente é um LLM com capacidades externas a mais.

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [10]:
agent_executor.invoke({"input": "crie um arquivo de texto contendo informações sobre o clima de Recife"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


chuvoso
Invoking: `write_file` with `{'file_name': 'clima_recife.txt', 'content': 'Informações sobre o clima de Recife:\n\nO clima atual em Recife é chuvoso.'}`


NoneO arquivo de texto contendo informações sobre o clima de Recife foi criado com sucesso. Ele contém a seguinte informação:

```
Informações sobre o clima de Recife:

O clima atual em Recife é chuvoso.
```

> Finished chain.


{'input': 'crie um arquivo de texto contendo informações sobre o clima de Recife',
 'output': 'O arquivo de texto contendo informações sobre o clima de Recife foi criado com sucesso. Ele contém a seguinte informação:\n\n```\nInformações sobre o clima de Recife:\n\nO clima atual em Recife é chuvoso.\n```'}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [11]:
agent_executor.invoke({"input": "quantas letras r existem na palavra strawberry?"})



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'word': 'strawberry', 'letter': 'r'}`


3Na palavra "strawberry", existem 3 letras "r".

> Finished chain.


{'input': 'quantas letras r existem na palavra strawberry?',
 'output': 'Na palavra "strawberry", existem 3 letras "r".'}